In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
import pandas as pd
import numpy as np
import psycopg2
import requests
import os
import re
import ast

In [2]:
headers = {'User-Agent': 'Mozilla/5.0',
           'From': 'itchyandscratchy@gmail.com'
          }

data_path = '/Users/amanda/Documents/Projects/insight/data/'

In [15]:
url = 'https://noc.esdc.gc.ca/Structure/Hierarchy/1a06c040a8bd4022ac498a80eb089bc1?objectid=%2Fd0IGA6qD8JPRfoj5UCjpg%3D%3D'

joblinks = []
# Main summary page
with requests.get(url, timeout=5, headers=headers) as u:     

    soup = BeautifulSoup(u.content)
    for link in soup.find_all('a'):
        joblinks.append(link.get('href'))
            
linkends = [s for s in joblinks if 'NocProfile' in s]            

In [17]:
print(linkends[0])

/Structure/NocProfile/3e3df8685e2e4b758474876abcd9c0e4?objectid=NmDF%2Bp5XjPpmMEzWh5irOIJPO4DWI3AoLXAGtO9E%2F%2Fr2xVJHKlh2Vvm4wy6CUIjy


In [67]:
base_url = 'https://noc.esdc.gc.ca'

overview_df = pd.DataFrame({'link':[],
                            'noc_code':[],
                            'job_group':[],
                            'description':[],
                            'alt_titles':[]})

duties_df = pd.DataFrame({'noc_code':[],
                          'duties':[],
                          'requirements':[]})

for link in linkends:

    # Main summary page
    url = base_url + link
    with requests.get(url, timeout=5, headers=headers) as u:     
        soup = BeautifulSoup(u.content) 
        #print(soup.prettify())

        # Main website content
        a = soup.find('main', attrs={'class':'container'})

        # Title and overview section
        title = a.find('h2',attrs={'style':'margin-top:25px;margin-bottom:0;'})
        intro = a.find('p', attrs={'class':'mrgn-tp-md'})
        
        if title is not None and intro is not None:
            noc_summary = title.text.split('–') #Note this is an en-dash, not a hyphen
            noc_code = noc_summary[0]
            job_group = noc_summary[-1]
            print(job_group)
            job_intro = intro.text.strip()
        else:
            continue
            
        title_sec = a.find('div',attrs={'id':'IndexTitles'})
        title_list = title_sec.findAll('li')
        alt_titles = [s.text.strip() for s in title_list]

        overview_df = overview_df.append({'link':link,
                                          'noc_code':noc_code,
                                          'job_group':job_group,
                                          'description':job_intro,
                                          'alt_titles':alt_titles}, ignore_index=True)
        
        # Duties and requirements
        job_duties = []
        duty_sec = soup.find('h4', text = 'Main duties').findNext('div', attrs={'class':'panel-body'})
        duty_list = duty_sec.findAll('li')
        if duty_list:
            job_duties = [s.text.strip() for s in duty_list]
        else:
            job_duties = 'NaN'
            
        job_req = []
        req_head = soup.find('h4', text = 'Employment requirements').findNext('ul')
        req_list = req_head.findAll('li')
        if req_list:
            job_reqs = [s.text.strip() for s in req_list]
        else:
            job_reqs = 'NaN'

        duties_df = duties_df.append({'noc_code':noc_code,
                                      'duties':job_duties,
                                      'requirements':job_reqs}, ignore_index=True)        
    
 

  Legislators
  Senior government managers and officials
  Senior managers - financial, communications and other business services
  Senior managers - health, education, social and community services and membership organizations
  Senior managers - trade, broadcasting and other services, n.e.c.
  Senior managers - construction, transportation, production and utilities
  Financial managers
  Human resources managers
  Purchasing managers
  Other administrative services managers
  Insurance, real estate and financial brokerage managers
  Banking, credit and other investment managers
  Advertising, marketing and public relations managers
  Other business services managers
  Telecommunication carriers managers
  Postal and courier services managers
  Engineering managers
  Architecture and science managers
  Computer and information systems managers
  Managers in health care
  Government managers - health and social policy development and program administration
  Government managers - econ

  Other assisting occupations in support of health services
  University professors and lecturers
  Post-secondary teaching and research assistants
  College and other vocational instructors
  Secondary school teachers
  Elementary school and kindergarten teachers
  Educational counsellors
  Judges
  Lawyers and Quebec notaries
  Psychologists
  Social workers
  Family, marriage and other related counsellors
  Professional occupations in religion
  Probation and parole officers and related occupations
  Employment counsellors
  Natural and applied science policy researchers, consultants and program officers
  Economists and economic policy researchers and analysts
  Business development officers and marketing researchers and consultants
  Social policy researchers, consultants and program officers
  Health policy researchers, consultants and program officers
  Education policy researchers, consultants and program officers
  Recreation, sports and fitness policy researchers, consultants

  Supervisors, logging and forestry
  Supervisors, mining and quarrying
  Contractors and supervisors, oil and gas drilling and services
  Underground production and development miners
  Oil and gas well drillers, servicers, testers and related workers
  Logging machinery operators
  Agricultural service contractors, farm supervisors and specialized livestock workers
  Contractors and supervisors, landscaping, grounds maintenance and horticulture services
  Fishing masters and officers
  Fishermen/women
  Underground mine service and support workers
  Oil and gas well drilling and related workers and services operators
  Chain saw and skidder operators
  Silviculture and forestry workers
  General farm workers
  Nursery and greenhouse workers
  Fishing vessel deckhands
  Trappers and hunters
  Harvesting labourers
  Landscaping and grounds maintenance labourers
  Aquaculture and marine harvest labourers
  Mine labourers
  Oil and gas drilling, servicing and related labourers
  Logging 

In [70]:
overview_df

,link,noc_code,job_group,description,alt_titles
0,/Structure/NocProfile/3e3df8685e2e4b758474876a...,0011,Legislators,Legislators participate in the activities of a...,"[Aboriginal band chief, Aboriginal band counci..."
1,/Structure/NocProfile/31784cb718a64da6928c5e7d...,0012,Senior government managers and officials,"Senior government managers and officials plan,...","[Ambassador, Assistant chief statistician - go..."
2,/Structure/NocProfile/801c2c84d93741aa9bec65bf...,0013,"Senior managers - financial, communications ...","Senior managers in financial, communications a...","[Advertising agency president, Assistant execu..."
3,/Structure/NocProfile/cd9cb97930984290947c34d2...,0014,"Senior managers - health, education, social ...","Senior managers in health, education, social a...","[Academic association president, Academic busi..."
4,/Structure/NocProfile/3f0e710b739d41b4a0aa8c42...,0015,"Senior managers - trade, broadcasting and ot...","Senior managers in trade, broadcasting and oth...","[Broadcasting corporation president, Casino ge..."
...,...,...,...,...,...
495,/Structure/NocProfile/10e157a46dbf4ff58df4da39...,9615,Labourers in rubber and plastic products man...,Labourers in rubber and plastic products manuf...,[Airbag extractor - rubber products manufactur...
496,/Structure/NocProfile/e5c335aef94c4450b1ed6534...,9616,Labourers in textile processing,Labourers in textile processing perform a vari...,"[Bale checker - textile processing, Battery lo..."
497,/Structure/NocProfile/abca92a16a7b498da09628df...,9617,Labourers in food and beverage processing,Labourers in food and beverage processing perf...,"[Animal feed sack filler, Bagger - food and be..."
498,/Structure/NocProfile/105c553c2f5841eb9f3aa4e5...,9618,Labourers in fish and seafood processing,Labourers in fish and seafood processing perfo...,"[Cannery labourer, Cannery labourer - fish pro..."


In [ ]:
# Try urls to find out if they exist
# Note this is a long process and only generally needs to be run once

#url_list = []

for i in range(1,6968):  
    try:
        url = 'https://www.jobbank.gc.ca/marketreport/summary-occupation/{}/ca'.format(i)  
        request = requests.get(url, headers=headers, timeout=5, allow_redirects=False)
        if request.status_code == 200:
            url_list.append(url)
            print(url)
    except:
        continue

https://www.jobbank.gc.ca/marketreport/summary-occupation/1/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/2/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/3/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/4/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/5/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/6/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/7/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/9/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/10/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/11/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/12/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/13/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/14/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/15/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/16/ca
https://www.jobbank.gc.ca/marketreport/summary-o

https://www.jobbank.gc.ca/marketreport/summary-occupation/131/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/132/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/133/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/134/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/135/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/136/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/137/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/138/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/139/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/140/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/141/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/142/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/143/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/144/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/145/ca
https://www.jobbank.gc.ca

https://www.jobbank.gc.ca/marketreport/summary-occupation/260/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/261/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/262/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/263/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/264/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/265/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/266/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/267/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/268/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/269/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/270/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/271/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/272/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/273/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/274/ca
https://www.jobbank.gc.ca

https://www.jobbank.gc.ca/marketreport/summary-occupation/419/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/422/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/424/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/426/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/427/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/428/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/429/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/430/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/431/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/432/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/433/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/434/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/435/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/436/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/437/ca
https://www.jobbank.gc.ca

https://www.jobbank.gc.ca/marketreport/summary-occupation/554/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/555/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/556/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/557/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/558/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/559/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/560/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/561/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/562/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/563/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/564/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/565/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/567/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/568/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/569/ca
https://www.jobbank.gc.ca

...

https://www.jobbank.gc.ca/marketreport/summary-occupation/840/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/841/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/842/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/843/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/844/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/845/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/846/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/847/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/848/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/849/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/850/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/851/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/852/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/853/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/854/ca
https://www.jobbank.gc.ca

https://www.jobbank.gc.ca/marketreport/summary-occupation/971/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/973/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/974/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/975/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/976/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/978/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/979/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/980/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/981/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/982/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/983/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/984/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/985/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/986/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/987/ca
https://www.jobbank.gc.ca

https://www.jobbank.gc.ca/marketreport/summary-occupation/1098/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1099/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1100/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1101/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1102/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1103/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1104/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1105/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1106/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1107/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1108/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1109/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1110/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1111/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1112/ca
https://ww

https://www.jobbank.gc.ca/marketreport/summary-occupation/1225/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1226/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1227/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1228/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1229/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1230/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1231/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1232/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1233/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1234/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1235/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1236/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1237/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1238/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1239/ca
https://ww

https://www.jobbank.gc.ca/marketreport/summary-occupation/1361/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1362/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1363/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1364/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1365/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1366/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1367/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1368/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1369/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1370/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1371/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1372/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1374/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1375/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/1376/ca
https://ww

In [11]:
url_list

[['https://www.jobbank.gc.ca/marketreport/summary-occupation/6969/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6970/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6971/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6972/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6973/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6974/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6975/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6976/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6977/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6978/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6979/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6980/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/6981/ca'],
 ['https://www.jobbank.gc.ca/marketreport/summary-occupation/698

In [ ]:
# Save url_list to file so I can load it later

file_path = (os.path.join(data_path,'processed','job_url_list.txt'))

with open(file_path, 'w') as filehandle:
    for listitem in url_list:
        filehandle.write('%s\n' % listitem)